In [0]:
dim_atleta = 'workspace.project_data_football_silver.dim_atleta'
dim_campeonato = 'workspace.project_data_football_silver.dim_campeonato'
dim_clube = 'workspace.project_data_football_silver.dim_clube'
dim_estadio = 'workspace.project_data_football_silver.dim_estadio'
dim_posicao = 'workspace.project_data_football_silver.dim_posicao'
dim_rodada = 'workspace.project_data_football_silver.dim_rodada'
fato_partida = 'workspace.project_data_football_silver.fato_partida'
fato_pontuacao = 'workspace.project_data_football_silver.fato_pontuacao'

In [0]:
spark.table(fato_partida).createOrReplaceTempView("tmp_silver_partida")
spark.table(dim_clube).createOrReplaceTempView("tmp_silver_clube")
spark.table(dim_campeonato).createOrReplaceTempView("tmp_silver_campeonato")

In [0]:
df_gold_classificacao_clubes = spark.sql("""

WITH base AS (

  -- Duas linhas por partida: mandante e visitante
  SELECT
    c.temporada,
    f.clube_casa_id AS clube_id,
    f.gols_casa AS gols_pro,
    f.gols_visitante AS gols_contra,
    -- Pontuação mandante
    CASE
      WHEN f.gols_casa > f.gols_visitante THEN 3
      WHEN f.gols_casa = f.gols_visitante THEN 1
      ELSE 0
    END AS pontos,
    -- Vitória mandante
    CASE WHEN f.gols_casa > f.gols_visitante THEN 1 ELSE 0 END AS vitoria,
    -- Empate mandante
    CASE WHEN f.gols_casa = f.gols_visitante THEN 1 ELSE 0 END AS empate
  FROM tmp_silver_partida f
    JOIN tmp_silver_campeonato c
      ON f.campeonato_id = c.campeonato_id

  UNION ALL

  -- Visitante: pontuação, vitória, empate
  SELECT
    c.temporada,
    f.clube_visitante_id AS clube_id,
    f.gols_visitante AS gols_pro,
    f.gols_casa AS gols_contra,
    CASE
      WHEN f.gols_visitante > f.gols_casa THEN 3
      WHEN f.gols_visitante = f.gols_casa THEN 1
      ELSE 0
    END AS pontos,
    CASE WHEN f.gols_visitante > f.gols_casa THEN 1 ELSE 0 END AS vitoria,
    CASE WHEN f.gols_visitante = f.gols_casa THEN 1 ELSE 0 END AS empate
  FROM tmp_silver_partida f
    JOIN tmp_silver_campeonato c
      ON f.campeonato_id = c.campeonato_id
),

classificacao AS (

  -- Consolida estatísticas por clube e temporada
  SELECT
    temporada,
    clube_id,
    COUNT(*) AS jogos, -- Jogos disputados
    SUM(vitoria) AS vitorias, -- Vitórias
    SUM(empate) AS empates, -- Empates
    COUNT(*) - SUM(vitoria) - SUM(empate) AS derrotas, -- Derrotas
    SUM(gols_pro) AS gols_pro, -- Gols marcados
    SUM(gols_contra) AS gols_contra, -- Gols sofridos
    SUM(gols_pro - gols_contra) AS saldo_gols, -- Saldo de gols
    SUM(pontos) AS pontos -- Pontos totais
  FROM base
  GROUP BY temporada, clube_id
),

ranking AS (

  -- Ranking dos clubes por temporada
  SELECT
    *,
    RANK() OVER (
      PARTITION BY temporada
      ORDER BY
        pontos DESC,
        vitorias DESC,
        saldo_gols DESC,
        gols_pro DESC
    ) AS posicao -- Posição na tabela
  FROM classificacao
)

-- Seleção final: classificação detalhada
SELECT DISTINCT
  r.temporada,
  r.posicao, -- Posição final

  r.clube_id,
  cl.nome_completo, -- Nome clube

  r.jogos,
  r.vitorias,
  r.empates,
  r.derrotas,
  r.gols_pro,
  r.gols_contra,
  r.saldo_gols,
  r.pontos,

  -- Aproveitamento em %
  ROUND((r.pontos * 100.0) / (r.jogos * 3), 2) AS aproveitamento_pct

FROM ranking r
LEFT JOIN tmp_silver_clube cl
  ON r.clube_id = cl.clube_id

ORDER BY
  r.temporada,
  r.posicao;
""")

#salvar em delta e salvar tabela
df_gold_classificacao_clubes.write\
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("workspace.project_data_football_gold.classificacao_clubes")
